In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:

import pandas as pd
import torch

from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torch import nn, optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, f1_score
from PIL import Image
import os
from sklearn.preprocessing import LabelEncoder
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
!unzip -j /content/drive/MyDrive/Datathonaİ/datasets/dataset/augmented_train.zip -d /content/augmented_train


Görüntülenen çıkış son 5000 satıra kısaltıldı.
  inflating: /content/augmented_train/image_14858.jpg  
  inflating: /content/augmented_train/flipped_image_14858.jpg  
  inflating: /content/augmented_train/image_10635.jpg  
  inflating: /content/augmented_train/flipped_image_10635.jpg  
  inflating: /content/augmented_train/image_16418.jpg  
  inflating: /content/augmented_train/flipped_image_16418.jpg  
  inflating: /content/augmented_train/image_16905.jpg  
  inflating: /content/augmented_train/flipped_image_16905.jpg  
  inflating: /content/augmented_train/image_12228.jpg  
  inflating: /content/augmented_train/flipped_image_12228.jpg  
  inflating: /content/augmented_train/image_15045.jpg  
  inflating: /content/augmented_train/flipped_image_15045.jpg  
  inflating: /content/augmented_train/image_14520.jpg  
  inflating: /content/augmented_train/flipped_image_14520.jpg  
  inflating: /content/augmented_train/image_14365.jpg  
  inflating: /content/augmented_train/flipped_image_14365

TRAİN CSV DOSYASINI COLAB HAFIZASINA YÜKLE UNUTMA

In [ ]:
import os
import pandas as pd
from PIL import Image, ImageEnhance
import random

# Dosya ve klasör yolları
images_dir = "/content/augmented_train"
train_csv = "/content/drive/MyDrive/Datathonaİ/datasets/dataset/augmented_train.csv"
augmented_dir = "/content/augmented_train"
augmented_csv = "/content/drive/MyDrive/Datathonaİ/datasets/dataset/augmented_trainbg.csv"

# augmented_train klasörünü oluştur
# augmented_train klasörünü oluştur
os.makedirs(augmented_dir, exist_ok=True)

# train.csv dosyasını yükle
train_df = pd.read_csv(train_csv)

# Augmentation işlemleri ve yeni CSV için liste
augmented_data = []

for index, row in train_df.iterrows():
    filename = row['filename']
    city = row['city']

    # Resmin tam yolunu oluştur
    image_path = os.path.join(images_dir, filename)

    # Görüntüyü yükle
    if os.path.exists(image_path):
        img = Image.open(image_path).convert("RGB")  # Renkli görüntü olduğundan emin olun

        # Orijinal resmi augmented_train klasörüne kaydet
        """
        original_save_path = os.path.join(augmented_dir, filename)
        img.save(original_save_path)
        """
        augmented_data.append({"filename": filename, "city": city})


        # Color Jitter uygulanmış görüntü
        brightness = ImageEnhance.Brightness(img).enhance(random.uniform(0.8, 1.2))
        contrast = ImageEnhance.Contrast(brightness).enhance(random.uniform(0.8, 1.2))
        saturation = ImageEnhance.Color(contrast).enhance(random.uniform(0.8, 1.2))
        jittered_img = ImageEnhance.Brightness(saturation).enhance(random.uniform(0.8, 1.2))

        jittered_filename = f"jittered_{filename}"
        jittered_save_path = os.path.join(augmented_dir, jittered_filename)
        jittered_img.save(jittered_save_path)
        augmented_data.append({"filename": jittered_filename, "city": city})


        # Rastgele kırpma ve yeniden boyutlandırma işlemi
        width, height = img.size
        crop_size = 540
        if width >= crop_size and height >= crop_size:
            left = random.randint(0, width - crop_size)
            top = random.randint(0, height - crop_size)
            cropped_img = img.crop((left, top, left + crop_size, top + crop_size))
            resized_img = cropped_img.resize((640, 640))

            cropped_filename = f"cropped_{filename}"
            cropped_save_path = os.path.join(augmented_dir, cropped_filename)
            resized_img.save(cropped_save_path)
            augmented_data.append({"filename": cropped_filename, "city": city})
        else:
            print(f"Warning: {filename} boyutları kırpma için uygun değil.")
    else:
        print(f"Warning: {filename} not found in {images_dir}")

# augmented_train.csv dosyasını oluştur
augmented_df = pd.DataFrame(augmented_data)
augmented_df.to_csv(augmented_csv, index=False)

print(f"Augmentation işlemi tamamlandı. Veriler {augmented_dir} klasörüne ve {augmented_csv} dosyasına kaydedildi.")


Augmentation işlemi tamamlandı. Veriler /content/augmented_train klasörüne ve /content/drive/MyDrive/Datathonaİ/datasets/dataset/augmented_trainbg.csv dosyasına kaydedildi.


In [ ]:

class CityDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]['filename']
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.dataframe.iloc[idx]['city']

        if self.transform:
            image = self.transform(image)

        return image, label

# CSV yükleme ve encoding

img_dir = "/content/augmented_train"


In [ ]:

label_mapping = {'Ankara': 0, 'Istanbul': 1, 'Izmir': 2}

#train_df, test_df = train_test_split(data, test_size=0.2, stratify=data['city'], random_state=42)

# Ayrılan verileri CSV olarak kaydetme
train_csv_path = "/content/augmented_trainbg.csv" #11k
test_csv_path = "/content/drive/MyDrive/Datathonaİ/datasets/dataset/testnonaugmnet.csv" #1400

train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

#train_df['city'] = train_df['city'].map(label_mapping)

##train_df.to_csv(train_csv_path, index=False)
#test_df.to_csv(test_csv_path, index=False)
# Dönüşümler
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])




In [ ]:
train_dataset = CityDataset(train_df, img_dir, transform=transform)
test_dataset = CityDataset(test_df, img_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Model oluşturuluyor
model = models.efficientnet_b3(pretrained=True)
num_classes = len(label_mapping)

# Classifier'ı değiştirme
model.classifier = nn.Sequential(
    nn.Dropout(p=0.3),  # Overfitting'i önlemek için Dropout
    nn.Linear(model.classifier[1].in_features, num_classes)
)

# Cihaz ayarı
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# **Katmanları Dondurma**
# İlk birkaç katmanı (örneğin, ilk 6 blok) dondur
# İlk dört katmanı dondur
for name, param in model.named_parameters():
    if "features.0" in name or "features.1" in name or "features.2" in name or "features.3" in name or "features.3" in name:
        param.requires_grad = False

# Eğitilecek parametreleri kontrol et
for name, param in model.named_parameters():
    print(f"{name}: {'Trainable' if param.requires_grad else 'Frozen'}")


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-b3899882.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-b3899882.pth
100%|██████████| 47.2M/47.2M [00:00<00:00, 180MB/s]


features.0.0.weight: Frozen
features.0.1.weight: Frozen
features.0.1.bias: Frozen
features.1.0.block.0.0.weight: Frozen
features.1.0.block.0.1.weight: Frozen
features.1.0.block.0.1.bias: Frozen
features.1.0.block.1.fc1.weight: Frozen
features.1.0.block.1.fc1.bias: Frozen
features.1.0.block.1.fc2.weight: Frozen
features.1.0.block.1.fc2.bias: Frozen
features.1.0.block.2.0.weight: Frozen
features.1.0.block.2.1.weight: Frozen
features.1.0.block.2.1.bias: Frozen
features.1.1.block.0.0.weight: Frozen
features.1.1.block.0.1.weight: Frozen
features.1.1.block.0.1.bias: Frozen
features.1.1.block.1.fc1.weight: Frozen
features.1.1.block.1.fc1.bias: Frozen
features.1.1.block.1.fc2.weight: Frozen
features.1.1.block.1.fc2.bias: Frozen
features.1.1.block.2.0.weight: Frozen
features.1.1.block.2.1.weight: Frozen
features.1.1.block.2.1.bias: Frozen
features.2.0.block.0.0.weight: Frozen
features.2.0.block.0.1.weight: Frozen
features.2.0.block.0.1.bias: Frozen
features.2.0.block.1.0.weight: Frozen
features

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
def save_checkpoint(epoch, model, optimizer, scheduler, path="/content/drive/MyDrive/Datathonaİ/datasets/dataset/efficentnet3.pth"):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()  # Scheduler'ın durumunu da kaydediyoruz
    }, path)


# Checkpoint yükleme fonksiyossnu
def load_checkpoint(model, optimizer, scheduler, path="/content/efficentnet2.pth"):
    if os.path.exists(path):
        checkpoint = torch.load(path, map_location=torch.device('cpu'))
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])  # Scheduler'ın durumunu yüklüyoruz
        start_epoch = checkpoint['epoch']
        print(f"Checkpoint yükleniyor: {path}, Epoch: {start_epoch}")
        return start_epoch
    else:
        print("Checkpoint bulunamadı. Eğitim sıfırdan başlıyor.")
        return 0


In [ ]:
def train_model(model, train_loader, criterion, optimizer, scheduler, epochs=50, checkpoint_interval=3, start_epoch=0):
    best_macro_f1 = 0.0
    for epoch in range(start_epoch, epochs):

        model.train()
        print("--------------------------------------")
        running_loss = 0.0
        i = 0
        for images, labels in train_loader:

            if (i+1) % 100 == 0:
                print(i)
            i += 1

            images, labels = images.to(device), labels.to(device).long()

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

        # Doğrulama
        model.eval()
        y_true, y_pred = [], []
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(preds.cpu().numpy())

        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None)
        for city, idx in label_mapping.items():
            print(f"Class: {city}, Precision: {precision[idx]:.4f}, Recall: {recall[idx]:.4f}, F1: {f1[idx]:.4f}")

        f1_macro = f1_score(y_true, y_pred, average='macro')
        print(f"Macro F1 Score: {f1_macro:.4f}")

        # Schedulerın adımı
        scheduler.step(f1_macro)

        if f1_macro > best_macro_f1:
            best_macro_f1 = f1_macro
            print(f"Yeni en iyi Macro F1 bulundu: {best_macro_f1:.4f}, modeli kaydediyorum...")
            save_checkpoint(epoch + 1, model, optimizer,scheduler)
        print("--------------------------------------")

# Checkpoint yükleme
start_epoch = load_checkpoint(model, optimizer, scheduler,  path="/content/efficentnet2.pth")


<ipython-input-22-ade29659d740>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device('cpu'))


Checkpoint yükleniyor: /content/efficentnet2.pth, Epoch: 1


In [ ]:
train_model(model, train_loader, criterion, optimizer, scheduler, start_epoch=start_epoch, epochs=50)

--------------------------------------
99
199
Epoch 2/50, Loss: 0.0460
Class: Ankara, Precision: 0.9211, Recall: 0.9153, F1: 0.9182
Class: Istanbul, Precision: 0.9061, Recall: 0.9353, F1: 0.9205
Class: Izmir, Precision: 0.9491, Recall: 0.9246, F1: 0.9367
Macro F1 Score: 0.9251
Yeni en iyi Macro F1 bulundu: 0.9251, modeli kaydediyorum...
--------------------------------------
--------------------------------------
99
199
Epoch 3/50, Loss: 0.0240
Class: Ankara, Precision: 0.9388, Recall: 0.8771, F1: 0.9069
Class: Istanbul, Precision: 0.9458, Recall: 0.9030, F1: 0.9239
Class: Izmir, Precision: 0.8682, Recall: 0.9655, F1: 0.9143
Macro F1 Score: 0.9150
--------------------------------------
--------------------------------------
99
199
Epoch 4/50, Loss: 0.0209
Class: Ankara, Precision: 0.9187, Recall: 0.9343, F1: 0.9265
Class: Istanbul, Precision: 0.9467, Recall: 0.9181, F1: 0.9322
Class: Izmir, Precision: 0.9298, Recall: 0.9418, F1: 0.9358
Macro F1 Score: 0.9315
Yeni en iyi Macro F1 bulund

In [ ]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())


precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None)
for city, idx in label_mapping.items():
    print(f"Class: {city}, Precision: {precision[idx]:.4f}, Recall: {recall[idx]:.4f}, F1: {f1[idx]:.4f}")

f1_macro = f1_score(y_true, y_pred, average='macro')
print(f"Macro F1 Score: {f1_macro:.4f}")

In [ ]:

.

In [ ]:
test2_csv_path = "/content/drive/MyDrive/Datathonaİ/datasets/dataset/test.csv"
test2_data = pd.read_csv(test2_csv_path)

test2_preds = []
test2_img_dir = "/content/drive/MyDrive/Datathonaİ/datasets/dataset/test/test/"

model.eval()
with torch.no_grad():
    for _, row in test2_data.iterrows():
        img_name = row['filename']
        img_path = os.path.join(test2_img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)

        outputs = model(image)
        _, preds = torch.max(outputs, 1)

        # Şehir ismini tahmin et
        predicted_city = [city for city, idx in label_mapping.items() if idx == preds.item()][0]
        test2_preds.append(predicted_city)

# Tahminleri kaydet
# Tahminleri 'city' kolonuna ekleyip, 'filename' ve 'city' formatında kaydet
test2_data['city'] = test2_preds
test2_data[['filename', 'city']].to_csv("/content/drive/MyDrive/Datathonaİ/datasets/dataset/testresnet_with_predictions.csv", index=False)
